# Turn a directory into something that can be easily visualized on a map!

In [1]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np;
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime

In [2]:
input_directory  = "/data/chime/geo/zone_a_full_contexual/stage2/"
output_directory = "/data/www/chime/movement-derivation/sandy_zone_a_contextual/"

In [3]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 81 users in /data/chime/geo/zone_a_full_contexual/stage2/


In [4]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        if t['geometry']:
            t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    
    df = pd.DataFrame(tweets)
    
    df = df.sort_values(by='date')
    
    first_date = df['date'][0]
    
    df['time'] = df['date'].apply(lambda x: datetime.datetime.strftime(x, '%Y-%m-%dT%H:%M:%SZ'))
    df['h']    = df['date'].apply(lambda x: int((x - first_date).total_seconds()/3600))
    df['m']    = df['date'].apply(lambda x: int((x - first_date).total_seconds()/60))
    if not q is None:
        q.put(1)
    return df

In [5]:
#debug
loader_function((users_in[0], input_directory, None))

,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user,time,h,m
0,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.8700896, 40.86819286]",2012-09-01 11:44:51+00:00,8,NaN,POINT (-73.8700896 40.86819286),1.0,NaN,Va a estar bueno @ My Dad's House http://t.co...,NaN,241864192762736640,79064816,3ltutuykt,2012-09-01T11:44:51Z,0,0
1,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 12:27:34+00:00,8,103.895704,POINT (-73.87029099999999 40.867271),NaN,0.040537,Nadaa (@ Home Sweet Home w/ 2 others) [pic]: h...,2563.0,241874940507344896,79064816,3ltutuykt,2012-09-01T12:27:34Z,0,42
2,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 12:29:57+00:00,8,0.000000,POINT (-73.87029099999999 40.867271),NaN,0.000000,GM @ Home Sweet Home http://t.co/xWXfCi8j,143.0,241875543102984192,79064816,3ltutuykt,2012-09-01T12:29:57Z,0,45
3,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.86887197, 40.86648697]",2012-09-01 12:51:20+00:00,8,147.779699,POINT (-73.86887197 40.86648697),NaN,0.115183,Mimiendo Daddy @ My home in my bed http://t.c...,1283.0,241880922729885697,79064816,3ltutuykt,2012-09-01T12:51:20Z,1,66
4,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 14:03:59+00:00,9,147.779699,POINT (-73.87029099999999 40.867271),NaN,0.033902,Patra pa lo galaxy...... Galasy s3 @ Home Swe...,4359.0,241899204308504576,79064816,3ltutuykt,2012-09-01T14:03:59Z,2,139
5,NaN,null,None,2012-09-01 14:07:03+00:00,9,NaN,NaN,NaN,NaN,La bb de Daddy http://t.co/y6skeJgb,-6725762.0,241899977692037120,79064816,3ltutuykt,2012-09-01T14:07:03Z,2,142
6,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 14:31:55+00:00,9,0.000000,POINT (-73.87029099999999 40.867271),NaN,0.000000,Todo para la voca @ Home Sweet Home http://t....,1676.0,241906234175926272,79064816,3ltutuykt,2012-09-01T14:31:55Z,2,167
7,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 14:38:21+00:00,9,0.000000,POINT (-73.87029099999999 40.867271),NaN,0.000000,Curandome con Donkey Kong @ Home Sweet Home h...,386.0,241907855555444737,79064816,3ltutuykt,2012-09-01T14:38:21Z,2,173
8,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 18:37:02+00:00,10,0.000000,POINT (-73.87029099999999 40.867271),NaN,0.000000,Princess sleeping @ Home Sweet Home http://t....,14321.0,241967918613676032,79064816,3ltutuykt,2012-09-01T18:37:02Z,6,412
9,1.0,"{""type"": ""Point"", ""coordinates"": [-73.86981887...","[-73.870291, 40.867271]",2012-09-01 18:54:43+00:00,10,0.000000,POINT (-73.87029099999999 40.867271),NaN,0.000000,Wife c voto hoy @ Home Sweet Home http://t.co...,1061.0,241972368975724545,79064816,3ltutuykt,2012-09-01T18:54:43Z,7,429


In [6]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 81, 100%

In [7]:
users[0].tail()

,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user,time,h,m
989,NaN,null,None,2012-12-01 03:14:14+00:00,6,NaN,NaN,NaN,NaN,#Summer2012 #A77 http://t.co/Ibk4A5mV,51.0,274712987741335552,79064816,3ltutuykt,2012-12-01T03:14:14Z,2175,130529
990,NaN,null,None,2012-12-01 03:15:06+00:00,6,NaN,NaN,NaN,NaN,#A77 http://t.co/0qbjaodR,52.0,274713203370512384,79064816,3ltutuykt,2012-12-01T03:15:06Z,2175,130530
991,NaN,null,None,2012-12-01 03:19:43+00:00,6,NaN,NaN,NaN,NaN,Shower time daddy #A77 http://t.co/xRjfomZ7,277.0,274714367675748352,79064816,3ltutuykt,2012-12-01T03:19:43Z,2175,130534
992,NaN,null,None,2012-12-01 03:21:43+00:00,6,NaN,NaN,NaN,NaN,La gorditaaa http://t.co/eLgq0T4F,120.0,274714867901022208,79064816,3ltutuykt,2012-12-01T03:21:43Z,2175,130536
993,NaN,null,None,2012-12-01 03:22:45+00:00,6,NaN,NaN,NaN,NaN,Lol #A77 http://t.co/SLXEjTcJ,62.0,274715128614752256,79064816,3ltutuykt,2012-12-01T03:22:45Z,2175,130537


# Write `geojson` files for the web

In [9]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [16]:
def write_geojson(args):
    u, output_directory, q = args
    file = u['user'][0].lower()
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in u.where(pd.notnull(u), None).iterrows():
        if pd.notnull(row.geometry):
            geom = mapping(row.geometry)
        else:
            geom = None
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':{'time':row['time'],
                                 'h':row['h'],
                                 'm':row['m'],
                                 'user':row['user'],
                                 'text':row['text'],
                                 's':row['speed'],
                                 'c':row['cluster']
                                 }
            }
        geojson['features'].append(feature)
    
    with open(output_directory+"/"+file+".geojson",'w') as oFile:
        json.dump(geojson,oFile)
        
    if q is not None:
        q.put(1)
    return 1

In [17]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, output_directory, q) for i in users]
result = p.map_async(write_geojson, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
result.get()
p.close()

Processed: 81, 100%